In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('C:/Users/Admin/Downloads/data/raw/VN_housing_dataset.csv')

cols_to_drop = ['Unnamed: 0', 'Ngày', 'Địa chỉ']
df = df.drop(columns=cols_to_drop)

df.columns = [
    'District', 'Ward', 'House_type', 'Legal', 'Floors', 'Bedrooms', 'Area', 'Length', 'Width', 'Price_per_m2'
]

# Hàm làm sạch số (loại bỏ chữ, đổi dấu phẩy thành chấm)
def extract_number(value):
    if pd.isna(value): return np.nan
    # Chuyển về chuỗi thường
    text = str(value).lower()
    # Nếu có dấu phẩy (86,96) thay bằng chấm (86.96)
    text = text.replace(',', '.')
    # Tìm số (bao gồm cả số thập phân)
    match = re.search(r"[-+]?\d*\.\d+|\d+", text)
    if match:
        return float(match.group())
    return np.nan

# Áp dụng cho các cột số
cols_to_clean = ['Area', 'Length', 'Width', 'Price_per_m2', 'Bedrooms', 'Floors']
for col in cols_to_clean:
    df[col] = df[col].apply(extract_number)

# Tính TỔNG GIÁ TRỊ CĂN NHÀ (Đây là cột quan trọng nhất để dự đoán)
# Công thức: Giá/m2 * Diện tích = Tổng giá (đơn vị: Triệu VNĐ)
df['Total_Price_Billion'] = (df['Price_per_m2'] * df['Area']) / 1000
# (Chia 1000 để đổi từ Triệu sang Tỷ cho số nhỏ dễ nhìn)

# Xử lý cột Quận/Huyện (Bỏ chữ 'Quận', 'Phường')
df['District'] = df['District'].str.replace('Quận', '').str.strip()
df['Ward'] = df['Ward'].str.replace('Phường', '').str.replace('Xã', '').str.strip()

# Xử lý thiếu (Missing Values)
# Xóa dòng nếu thiếu Diện tích hoặc Giá (vì không train được)
df = df.dropna(subset=['Area', 'Total_Price_Billion'])

# Cột Pháp lý: Điền 'Dang_cap_nhat'
df['Legal'] = df['Legal'].fillna('Dang_cap_nhat')

# Cột Số tầng, Phòng ngủ: Điền bằng trung vị (Median)
df['Floors'] = df['Floors'].fillna(df['Floors'].median())
df['Bedrooms'] = df['Bedrooms'].fillna(df['Bedrooms'].median())

# Xóa cột Dài, Rộng (vì thiếu quá nhiều)
df = df.drop(columns=['Length', 'Width'])

# Lọc theo Diện tích
# Chỉ lấy nhà từ 10m2 đến 500m2
df = df[(df['Area'] >= 10) & (df['Area'] <= 500)]

# Lọc bỏ số phòng ngủ
# Nếu diện tích < 40m2 mà có >= 8 phòng ngủ -> XÓA (Vì quá chật)
df = df[~((df['Area'] < 40) & (df['Bedrooms'] >= 8))]

# Nếu số tầng < 2 mà có >= 5 phòng ngủ (trừ khi diện tích > 100m2) -> XÓA
df = df[~((df['Floors'] < 2) & (df['Bedrooms'] >= 5) & (df['Area'] < 100))]

# Lọc theo Giá
# Chỉ lấy nhà giá từ 500 Triệu (0.5 tỷ) đến 100 Tỷ
df = df[(df['Total_Price_Billion'] >= 0.5) & (df['Total_Price_Billion'] <= 100)]

# Lưu file sạch



In [47]:
df.count()

District               81824
Ward                   81786
House_type             81795
Legal                  81825
Floors                 81825
Bedrooms               81825
Area                   81825
Price_per_m2           81825
Total_Price_Billion    81825
dtype: int64

In [48]:
df.head(10)

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4.0,5.0,46.0,86.96,4.00016
1,Thanh Xuân,Kim Giang,"Nhà mặt phố, mặt tiền",Dang_cap_nhat,5.0,3.0,37.0,116.22,4.30014
2,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,65.00,2.60000
3,Tây Hồ,Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,5.0,6.0,51.0,100.00,5.10000
4,Thanh Xuân,Kim Giang,"Nhà ngõ, hẻm",Dang_cap_nhat,5.0,4.0,36.0,86.11,3.09996
5,Cầu Giấy,Yên Hoà,"Nhà ngõ, hẻm",Đã có sổ,5.0,10.0,46.0,104.35,4.80010
6,Đống Đa,Trung Liệt,"Nhà ngõ, hẻm",Dang_cap_nhat,5.0,3.0,52.0,112.50,5.85000
7,Hai Bà Trưng,Đống Mác,"Nhà mặt phố, mặt tiền",Đã có sổ,6.0,5.0,32.0,184.38,5.90016
8,Tây Hồ,Xuân La,"Nhà ngõ, hẻm",Dang_cap_nhat,5.0,4.0,75.0,120.00,9.00000
9,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ,4.0,3.0,41.0,64.63,2.64983


In [49]:
# 1. Lưu và mã hóa District
df['District_old'] = df['District'].copy()
df['District'] = pd.factorize(df['District'])[0]

print("--- BẢNG TRA CỨU DISTRICT ---")
district_map = df[['District_old', 'District']].drop_duplicates().sort_values('District')
print(district_map.to_string(index=False)) # Dùng to_string để hiện thị đẹp hơn

--- BẢNG TRA CỨU DISTRICT ---
    District_old  District
             NaN        -1
        Cầu Giấy         0
      Thanh Xuân         1
    Hai Bà Trưng         2
          Tây Hồ         3
         Đống Đa         4
         Hà Đông         5
 Huyện Thanh Trì         6
       Hoàng Mai         7
       Long Biên         8
       Hoàn Kiếm         9
     Nam Từ Liêm        10
         Ba Đình        11
  Huyện Hoài Đức        12
     Bắc Từ Liêm        13
Huyện Đan Phượng        14
 Huyện Thanh Oai        15
   Huyện Sóc Sơn        16
   Huyện Gia Lâm        17
 Huyện Chương Mỹ        18
  Huyện Đông Anh        19
Huyện Thường Tín        20
  Thị xã Sơn Tây        21
   Huyện Mê Linh        22
  Huyện Quốc Oai        23
  Huyện Phúc Thọ        24
 Huyện Phú Xuyên        25
     Huyện Ba Vì        26
Huyện Thạch Thất        27
    Huyện Mỹ Đức        28


In [50]:
# 2. Lưu và mã hóa Ward
df['Ward_old'] = df['Ward'].copy()
df['Ward'] = pd.factorize(df['Ward'])[0]

print("--- BẢNG TRA CỨU WARD ---")
ward_map = df[['Ward_old', 'Ward']].drop_duplicates().sort_values('Ward')
print(ward_map.to_string(index=False))

--- BẢNG TRA CỨU WARD ---
           Ward_old  Ward
                NaN    -1
           Nghĩa Đô     0
          Kim Giang     1
          Minh Khai     2
          Thụy Khuê     3
            Yên Hoà     4
         Trung Liệt     5
           Đống Mác     6
            Xuân La     7
           Văn Quán     8
  Thị trấn Văn Điển     9
          Định Công    10
              Bồ Đề    11
        Quang Trung    12
        Thanh Lương    13
       Khương Trung    14
           Phúc Tân    15
           Gia Thụy    16
        Khương Đình    17
        Phương Canh    18
          Tương Mai    19
             La Khê    20
             Mễ Trì    21
         Khương Mai    22
            Láng Hạ    23
           Quan Hoa    24
             Tây Mỗ    25
         Ngọc Khánh    26
             Đại Mỗ    27
       Tả Thanh Oai    28
          Mỹ Đình 1    29
           Tam Hiệp    30
            Cống Vị    31
          Bách Khoa    32
          Vĩnh Phúc    33
          Kim Chung    34
            

In [51]:
# 3. Lưu và mã hóa House_type
df['House_type_old'] = df['House_type'].copy()
df['House_type'] = pd.factorize(df['House_type'])[0]

print("--- BẢNG TRA CỨU HOUSE_TYPE ---")
house_type_map = df[['House_type_old', 'House_type']].drop_duplicates().sort_values('House_type')
print(house_type_map.to_string(index=False))

--- BẢNG TRA CỨU HOUSE_TYPE ---
       House_type_old  House_type
                  NaN          -1
         Nhà ngõ, hẻm           0
Nhà mặt phố, mặt tiền           1
         Nhà biệt thự           2
      Nhà phố liền kề           3


In [52]:
# 4. Lưu và mã hóa Legal
df['Legal_old'] = df['Legal'].copy()
df['Legal'] = pd.factorize(df['Legal'])[0]

print("--- BẢNG TRA CỨU LEGAL ---")
legal_map = df[['Legal_old', 'Legal']].drop_duplicates().sort_values('Legal')
print(legal_map.to_string(index=False))


--- BẢNG TRA CỨU LEGAL ---
    Legal_old  Legal
     Đã có sổ      0
Dang_cap_nhat      1
  Đang chờ sổ      2
 Giấy tờ khác      3


In [53]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion,District_old,Ward_old,House_type_old,Legal_old
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ
1,1,1,1,1,5.0,3.0,37.0,116.22,4.30014,Thanh Xuân,Kim Giang,"Nhà mặt phố, mặt tiền",Dang_cap_nhat
2,2,2,0,0,4.0,4.0,40.0,65.00,2.60000,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ
3,3,3,0,0,5.0,6.0,51.0,100.00,5.10000,Tây Hồ,Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ
4,1,1,0,1,5.0,4.0,36.0,86.11,3.09996,Thanh Xuân,Kim Giang,"Nhà ngõ, hẻm",Dang_cap_nhat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82491,13,57,3,1,5.0,3.0,38.0,81.58,3.10004,Bắc Từ Liêm,Phúc Diễn,Nhà phố liền kề,Dang_cap_nhat
82492,0,24,1,1,5.0,3.0,50.0,292.00,14.60000,Cầu Giấy,Quan Hoa,"Nhà mặt phố, mặt tiền",Dang_cap_nhat
82493,0,24,1,0,5.0,4.0,41.0,341.46,13.99986,Cầu Giấy,Quan Hoa,"Nhà mặt phố, mặt tiền",Đã có sổ
82494,0,24,0,0,5.0,4.0,60.0,101.67,6.10020,Cầu Giấy,Quan Hoa,"Nhà ngõ, hẻm",Đã có sổ


In [54]:
df = df.drop(columns=['District_old', 'Ward_old', 'House_type_old', 'Legal_old'])

In [55]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016
1,1,1,1,1,5.0,3.0,37.0,116.22,4.30014
2,2,2,0,0,4.0,4.0,40.0,65.00,2.60000
3,3,3,0,0,5.0,6.0,51.0,100.00,5.10000
4,1,1,0,1,5.0,4.0,36.0,86.11,3.09996
...,...,...,...,...,...,...,...,...,...
82491,13,57,3,1,5.0,3.0,38.0,81.58,3.10004
82492,0,24,1,1,5.0,3.0,50.0,292.00,14.60000
82493,0,24,1,0,5.0,4.0,41.0,341.46,13.99986
82494,0,24,0,0,5.0,4.0,60.0,101.67,6.10020


In [56]:
df.to_csv('C:/Users/Admin/Downloads/data/processed/clean_vn_housing.csv', index=False)